# Conversion of MODIS File Format by Spherical Coordinates
---

## Introduction

This program processes an HDF file and extract the data of a SDS of your choice. After having loaded the data file, it lists all the SDS available in it. You can pick the one of your interest and proceed.

The notebook also gives you a chance to look into the data, remove the fill values, includes a function to output the values of a coordinate from signed decision to degree/minutes/seconds format ( *_you might not even need it_*).

**Ensure when running this notebook**
* You have Anaconda and Python installed on your computer.
* You launch Jupyter NOT from the start-menu but from an Anaconda Prompt terminal.
* You launch the Jupyter notebook from within the folder where it located -
    - change directory in Anaconda Prompt using **cd** commands until you are where the *.ipnyb* file is located
    - the data file is located right there or in a folder called *data*

### _Note:_
<i>
1. Every block of code in this notebook is logically grouped
2. The purpose of each block is briefed just before the block itself
3. The clarity of the code and other technical explanations are included in the code as comments
</i>

#### Import Libraries
Import all the necessary libraries that would be required for this project.
SD is the library that is capable of handling HDF files generated by MODIS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap, cm

from pyhdf import SD


If the above piece of code fail that would mean that you do not have all the libraries propoerly installed. Check the following: 
* The library itself was never installed. Install them using conda install command.
* You have installed the libraries but they are not picked up by Jupyter environment. Fix your sys.path and kernal.
* You did not launch Jupyter using 'jupyter notebook' command from within the folder where this notebook is located.

#### Load the HDF File
- Insert the filename in the code below that needs processing. Ensure the relative path of the file from the location of this notebook.
- Currently this code is capable of loading one file at a time.

In [26]:
# Insert your data file name between the quotes in the statement below. 
# Ensure that the file is located in a directory called 'data'

try:
    FILE_NAME = 'MYD04_L2.A2017249.2105.006.2017250160535.hdf'
    hdf = SD.SD('../data/{}'.format(FILE_NAME))

except:
    print('There is a problem with the file format. Can you ensure that the HDF file that you have loaded is not corrupt.')
    print('Try loading the HDF file using Panoply, if needed.')

#### Latitudes and Longitudes
- The complete set of latitudes and longitudes for all the data points are available within the HDF file. They can be extracted in 2 different arrays.
- Determine the start and final lat-long and check if they are right.

In [11]:
# Find out the starting and ending lat-long for this data set
lat = hdf.select('Latitude')
lon = hdf.select('Longitude')
    
latitudes = lat[:]
longitudes = lon[:]

edgeCoordinates = [[latitudes.min(),  longitudes.min()], [latitudes.max(), longitudes.max()]]

print(edgeCoordinates)

[[31.052055, -138.71803], [52.295116, -105.552605]]


#### Dataset List
- The HDF file may contain a number of datasets. It is best to list them down so that you can pick the right one.
- Alternatively, load this HDF file on PanoPly and locate the dataset.

In [5]:
datasets = hdf.datasets()
sdsList = []

for i, v in enumerate(datasets):
    sdsList += [v]    # extend, not append
    print('{}. {}'.format(i, v))

sdsCount = i + 1


0. Longitude
1. Latitude
2. Scan_Start_Time
3. Solar_Zenith
4. Solar_Azimuth
5. Sensor_Zenith
6. Sensor_Azimuth
7. Scattering_Angle
8. Land_sea_Flag
9. Aerosol_Cldmask_Land_Ocean
10. Cloud_Pixel_Distance_Land_Ocean
11. Land_Ocean_Quality_Flag
12. Optical_Depth_Land_And_Ocean
13. Image_Optical_Depth_Land_And_Ocean
14. Average_Cloud_Pixel_Distance_Land_Ocean
15. Aerosol_Type_Land
16. Fitting_Error_Land
17. Surface_Reflectance_Land
18. Corrected_Optical_Depth_Land
19. Corrected_Optical_Depth_Land_wav2p1
20. Optical_Depth_Ratio_Small_Land
21. Number_Pixels_Used_Land
22. Mean_Reflectance_Land
23. STD_Reflectance_Land
24. Mass_Concentration_Land
25. Aerosol_Cloud_Fraction_Land
26. Quality_Assurance_Land
27. Solution_Index_Ocean_Small
28. Solution_Index_Ocean_Large
29. Effective_Optical_Depth_Best_Ocean
30. Effective_Optical_Depth_Average_Ocean
31. Optical_Depth_Small_Best_Ocean
32. Optical_Depth_Small_Average_Ocean
33. Optical_Depth_Large_Best_Ocean
34. Optical_Depth_Large_Average_Ocean
35. 

#### Extract the Data
- Replace the name of the dataset below to the one of your interest.
- The dataset itself could either be 2 dimensional or be 3-dimensional. The below code is generalised for the kinds.

In [6]:
sds = hdf.select('Corrected_Optical_Depth_Land')
data = sds.get()

# If the data is 3D then the size of outermost list is the 'depth'; in case of 2D, it is the row
if data.ndim == 2:
    nRows = len(data)
    nColumns = len(data[0])
else:
    nDepth = len(data)
    nRows = len(data[0])
    nColumns = len(data[0][0])
               
print('No of rows {} and no of columns {}'.format(nRows, nColumns))
if data.ndim == 3:
    print('Also, there are {} layers of data available for each lat-long'.format(nDepth))


No of rows 203 and no of columns 135
Also, there are 3 layers of data available for each lat-long


#### Clean the Data
- Replace the fill values
- Apply the scale factor

In [9]:
# Replace the fill-values (the missing data points) with NaN
attrs = sds.attributes(full=1)
fillvalue=attrs['_FillValue']
fv = fillvalue[0]

# Turn fillvalues to NaN
data=data.astype(float)
data[data == fv] = np.nan

# Apply the scaleFactor
attributes=sds.attributes()
scaleFactor=attributes['scale_factor']
print('The scale factor of this data is {:.3f}.'.format(scaleFactor))

scaledData = data * scaleFactor

print('The top 10 rows of your raw data look like this. It shows only the first 10 of the total {} columns.'.format(nColumns))
print(scaledData[0:10, 0:10] if scaledData.ndim == 2 else scaledData[:, 0:10, 0:10])



The scale factor of this data is 0.001.
The top 10 rows of your raw data look like this. It shows only the first 10 of the total 135 columns.
[[[0.25900001 0.27600001 0.28400001 0.37700002 0.49200002        nan
          nan        nan        nan        nan]
  [0.23200001 0.19000001 0.25600001 0.33000002 0.44700002        nan
          nan        nan        nan        nan]
  [0.22400001 0.15800001 0.24500001 0.34100002        nan        nan
          nan        nan 0.44800002 0.42100002]
  [0.27200001 0.15600001 0.22100001 0.34300002        nan        nan
   0.42300002        nan 0.41200002 0.40700002]
  [0.28100001 0.13300001 0.20300001 0.27900001        nan        nan
          nan 0.42000002 0.33400002 0.36200002]
  [0.28100001 0.15500001 0.24600001 0.28300001 0.38600002        nan
          nan 0.43000002 0.30800001 0.35400002]
  [0.27400001 0.20600001 0.23900001 0.26200001 0.37600002        nan
          nan        nan 0.30700001 0.32500002]
  [0.28000001 0.22400001 0.33400002 0.3

#### Crop the Data
- The downloaded data could be much more than what you might be interested in. Enter the start and final lat-long coordinates.
- If you ignore this step then the entire dataset will be processed and exported to the final output file.


In [23]:
# Please uncomment and enter the new start lat-long, separated by a space (e.g., 23.208 88.762).

edgeCoordinates[0] = [ 45.0, -125.0 ]

lat1 = float(edgeCoordinates[0][0])
lon1 = float(edgeCoordinates[0][1])

# Please uncomment enter the new final lat-long, separated by a space (e.g., 35.812 92.253)

edgeCoordinates[1] = [ 50.0, -115.0 ]

lat2 = float(edgeCoordinates[1][0])
lon2 = float(edgeCoordinates[1][1])


print("{}, {} - {}, {}".format(lat1, lon1, lat2, lon2))

45.0, -125.0 - 50.0, -115.0


#### Output Format
- The datapoints will be inserted into an array with each updated with its latitude and longitude values

In [24]:
outputTable = []

if lat1 < lat2:
    minLat, maxLat = lat1, lat2
else:
    minLat, maxLat = lat2, lat1

if lon1 < lon2:
    minLon, maxLon = lon1, lon2
else:
    minLon, maxLon = lon2, lon1

if scaledData.ndim == 2:
    for i in range(nRows):
        for j in range(nColumns):
            if latitudes[i,j] >= minLat and latitudes[i,j] <= maxLat:
                if longitudes[i,j] >= minLon and longitudes[i,j] <= maxLon:
                    outputTable.append([latitudes[i,j], longitudes[i,j], scaledData[i,j]])
else:
    # tempList = []
    for i in range(nRows):
        for j in range(nColumns):
             if latitudes[i,j] >= minLat and latitudes[i,j] <= maxLat:
                if longitudes[i,j] >= minLon and longitudes[i,j] <= maxLon:

                    tempList = [latitudes[i,j], longitudes[i,j]]

                    isRowNan = True # set to true but will become false if minimum one item in the row is a valid number
                    for k in range(nDepth):
                        value = scaledData[k, i, j]
                        isRowNan = False if not np.isnan(value) else isRowNan
                        tempList += [value]     # the original data comes with depth as the 1st level of hierarchy
                    if not isRowNan:
                        outputTable.append(tempList)

print(outputTable[:10])

[[45.042225, -115.47438, 1.9620000931899995, 1.5830000751884654, 1.2790000607492402], [45.025204, -115.6595, 2.1410001016920432, 1.7400000826455653, 1.4140000671613961], [45.007553, -115.84765, 2.6220001245383173, 2.145000101882033, 1.7530000832630321], [45.130497, -115.49281, 1.7650000838330016, 1.4080000668764114, 1.1250000534346327], [45.112797, -115.68523, 3.011000143014826, 2.462000116938725, 2.012000095564872], [45.09523, -115.87281, 2.4340001156087965, 1.9910000945674255, 1.6270000772783533], [45.078484, -116.048904, 2.499000118696131, 2.0440000970847905, 1.6710000793682411], [45.06124, -116.22701, 2.331000110716559, 1.9070000905776396, 1.5590000740485266], [45.044327, -116.399055, 2.4360001157037914, 1.9930000946624205, 1.6290000773733482], [45.02709, -116.57148, 2.1000000997446477, 1.7030000808881596, 1.387000065878965]]


#### Save in an Output File


In [29]:
import csv

OUTPUT_FILE = '../data/{}.csv'.format(FILE_NAME[:-4])

with open(OUTPUT_FILE, 'w', newline = '') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     for line in outputTable:
         wr.writerow(line)

print('Done')